In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the common data path
data_path = '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone'
train_data_dir = os.path.join(data_path, 'healthy')  # This will just be the directory containing images
val_data_dir = os.path.join(data_path, 'diseased')  # Update accordingly

In [4]:
# Data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)


In [5]:
# For validation data, only rescale
val_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
import os

# Check for the exact path in Kaggle
base_path = "/kaggle/input/"

# List available datasets or files in the base path
print(os.listdir(base_path))


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/'

In [7]:
# Check the directory structure
import os

# Define the common data path
data_path = '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone'

# Print the contents to verify the structure
print("Data Path Contents:")
print(os.listdir(data_path))

# Define the training and validation directories based on structure
train_data_dir = data_path  # If all images are in this folder
val_data_dir = data_path  # Adjust if you have a different folder structure

Data Path Contents:


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone'

In [7]:
# Load training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(150, 150),  # Adjust based on your image size
    batch_size=32,
    class_mode='binary'  # Use 'categorical' for multi-class classification
)

Found 12446 images belonging to 1 classes.


In [8]:
# Load validation data
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(150, 150),  # Adjust based on your image size
    batch_size=32,
    class_mode='binary'
)

Found 12446 images belonging to 1 classes.


In [9]:
# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Change to softmax for multi-class
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
# Train the model
model.fit(train_generator, validation_data=val_generator, epochs=5)

# Save the model
model.save('kidney_disease_model.h5')

Epoch 1/5


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


389/389 ━━━━━━━━━━━━━━━━━━━━ 502s 1s/step - accuracy: 0.9869 - loss: 0.0125 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
389/389 ━━━━━━━━━━━━━━━━━━━━ 458s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
389/389 ━━━━━━━━━━━━━━━━━━━━ 454s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
389/389 ━━━━━━━━━━━━━━━━━━━━ 499s 1s/step - accuracy: 1.0000 - loss: 8.7865e-37 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
389/389 ━━━━━━━━━━━━━━━━━━━━ 454s 1s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [12]:
# Loading the Trained Model

In [13]:
model = tf.keras.models.load_model('kidney_disease_model.h5')


In [17]:
def preprocess_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match training data scaling (if applicable)
    return img_array



In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('kidney_disease_model.h5')

# Function to preprocess the image
def preprocess_image(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)  # Resize to target size
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize the image to the range [0, 1]
    return img_array

# Path to the new image
img_path = '/kaggle/input/ct-kidney-dataset-normal-cyst-tumor-and-stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Tumor/Tumor- (1000).jpg'  # Replace with the actual image path

# Preprocess the image (ensure target_size matches the training input size)
preprocessed_img = preprocess_image(img_path, target_size=(150, 150))  # Use 150x150 as per your model's input layer

# Make prediction
prediction = model.predict(preprocessed_img)

# Print the predicted class
predicted_class = np.argmax(prediction, axis=1)
print(f'Predicted class: {predicted_class}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Predicted class: [0]


In [ ]:
# from tensorflow.keras.preprocessing import image
# import numpy as np

# # Load and preprocess the image
# def preprocess_image(img_path):
#     img = image.load_img(img_path, target_size=(150, 150))  # Resize to match training size
#     img_array = image.img_to_array(img)  # Convert image to array
#     img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
#     img_array = img_array / 255.0  # Rescale as done in training
#     return img_array

In [1]:
#  Using a pretrined Model 

# Different Approach  out Of Curiosity 😃😃😃⚡🚀❌
# Failed Heavy Computational Task 😅😅😅

In [23]:
# from tensorflow.keras.applications import MobileNetV2
# from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
# from tensorflow.keras.models import Model

# # Load the pretrained MobileNetV2 model, excluding the top layers
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# # Freeze the layers of the base model so that they're not trained
# for layer in base_model.layers:
#     layer.trainable = False

# # Add new layers on top of the base model
# x = base_model.output
# x = GlobalAveragePooling2D()(x)  # Global pooling instead of flattening
# x = Dense(1024, activation='relu')(x)  # Fully connected layer
# predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

# # Create the new model
# model = Model(inputs=base_model.input, outputs=predictions)

# # Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
# from tensorflow.keras.callbacks import ModelCheckpoint

# # Create a callback that saves the model's weights
# checkpoint = ModelCheckpoint('kidney_disease_model.h5', save_best_only=True)

# # Train the model with checkpoints
# model.fit(train_generator, validation_data=val_generator, epochs=5, callbacks=[checkpoint])


In [25]:
# # Train the model
# model.fit(train_generator, validation_data=val_generator, epochs=5)

# # Save the model
# model.save('kidney_disease_model.h5')


In [26]:
# model = Sequential([
#     Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
#     MaxPooling2D(2, 2),
#     Conv2D(32, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(2, 2),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(1, activation='sigmoid')  # Output for binary classification
# ])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [27]:
# # Change target size in train_generator and val_generator
# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(128, 128),
#     batch_size=32,
#     class_mode='binary'
# )

# val_generator = val_datagen.flow_from_directory(
#     val_data_dir,
#     target_size=(128, 128),
#     batch_size=32,
#     class_mode='binary'
# )


In [28]:
# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(150, 150),
#     batch_size=16,  # Smaller batch size
#     class_mode='binary'
# )


In [29]:
# # Reduce the number of epochs if it trains too long
# model.fit(train_generator, validation_data=val_generator, epochs=3)


In [30]:
# # Early Stopping 


# from tensorflow.keras.callbacks import EarlyStopping

# # Add Early Stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# # Train with early stopping
# model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[early_stopping])


In [31]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
# from tensorflow.keras.models import Model

# # Load the pre-trained ResNet50 model without the top classification layers
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# # Freeze the base model's layers so they don't get updated during the first training stage
# for layer in base_model.layers:
#     layer.trainable = False

In [32]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("object-detection", model="facebook/detr-resnet-50")

In [33]:
# from transformers import AutoModel

# model = AutoModel.from_pretrained("facebook/detr-resnet-50", local_files_only=True)


In [ ]:
# This is in working Mode or under construction Model